# The Transaction-Level TinyALU
 The TinyALU TLM DUT

In [1]:
from pyuvm import *
from tb_pkg import *
import random

class PythonProxy(uvm_component):
    @staticmethod
    def alu_op(A, B, op):
        assert isinstance(op, Ops), "The tinyalu op must be of type ops"
        if op == Ops.ADD:
            result = A + B
        elif op == Ops.AND:
            result = A & B
        elif op == Ops.XOR:
            result = A ^ B
        elif op == Ops.MUL:
            result = A * B
        time.sleep(0.1)  # Takes time as a simulation would.
        return result
    
    def build_phase(self):
        # The FIFOs
        self.stim_f = uvm_tlm_fifo("stim_f", self)
        self.cmd_f = uvm_tlm_analysis_fifo("cmd_f", self)
        self.result_f = uvm_tlm_analysis_fifo("result_f", self)
        
        # The Stimulus Ports (for send_op())
        self.stim_put = uvm_put_port("stim_put", self)
        self.stim_get = uvm_get_port("stim_get", self)
        
        # The Command Ports (for get_cmd())
        self.cmd_put = uvm_put_port("cmd_put", self)
        self.cmd_get = uvm_get_port("cmd_get", self)
       
        # The Result Ports (for get_result())
        self.result_put = uvm_put_port("result_put",self)
        self.result_get = uvm_get_port("result_get", self)

    def connect_phase(self):
        self.stim_put.connect(self.stim_f.put_export)
        self.stim_get.connect(self.stim_f.get_export)
        
        self.cmd_put.connect(self.cmd_f.put_export)
        self.cmd_get.connect(self.cmd_f.get_export)
        
        self.result_put.connect(self.result_f.put_export)
        self.result_get.connect(self.result_f.get_export)

    def send_op(self, op):
        self.stim_put.put(op)
    
    def get_cmd(self):
        cmd  = self.cmd_get.get()
        return cmd
    
    def get_result(self):
        result = self.result_get.get()
        return result        
        
    def run_phase(self):
        while True:
            cmd = self.stim_get.get()
            result = self.alu_op(cmd.A, cmd.B, cmd.op)
            result_txn = AluResult("result_txn", result)
            self.cmd_put.put(cmd)
            self.result_put.put(result_txn)

# The code below is not in the book. It simply tests the code above.
            
class TxnPrinter(uvm_subscriber):
    def write(self, txn):
        self.logger.info(txn)
    

class Monitor(uvm_component):
    def __init__(self, name, parent, dut, method_name):
        super().__init__(name, parent)
        self.dut = dut
        self.method_name = method_name
    
    def build_phase(self):
        self.analysis_port = uvm_analysis_port("analysis_port", self)

    def run_phase(self):
        while True:
            get_method = getattr(self.dut, self.method_name)
            result = get_method()
            self.analysis_port.write(result)    
        
    
class AluEnv(uvm_env):
    def __init__(self, name, parent,dut):
        super().__init__(name, parent)
        self.dut = dut
        
    def build_phase(self):
        self.cmd_mon = Monitor("cmd_mon", self, self.dut, "get_cmd")
        self.rslt_mon = Monitor("rslt_mon", self, self.dut, "get_result")
        self.txn_printer = TxnPrinter("txn_printer", self)
        
    def connect_phase(self):
        self.cmd_mon.analysis_port.connect(self.txn_printer)
        self.rslt_mon.analysis_port.connect(self.txn_printer)

class AluTest(uvm_test):
    def build_phase(self):
        self.dut = PythonProxy("dut", self)
        self.env = AluEnv("env", self, self.dut)
        
    def run_phase(self):
        self.raise_objection()
        for _ in range(5):
            A = random.randrange(256)
            B = random.randrange(256)
            op = random.choice(list(Ops))
            cmd = AluCommand("cmd", A, B, op)
            self.dut.send_op(cmd)
        time.sleep(0.6) # Wait for last operation to complete
        self.drop_objection()

uvm_root().run_test("AluTest")
        

INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: cmd : A: 0xfb OP: XOR (3) B: 0x34
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: result_txn: 0x00cf
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: cmd : A: 0x3e OP: XOR (3) B: 0x0b
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: result_txn: 0x0035
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: result_txn: 0x0127
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: cmd : A: 0xfa OP: ADD (1) B: 0x2d
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: cmd : A: 0x16 OP: MUL (4) B: 0xb3
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: result_txn: 0x0f62
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: result_txn: 0x12de
INFO: <ipython-input-1-036cb62c15a0>(71)[uvm_test_top.env.txn_printer]: cmd : A: 0x69 OP: MUL (4) B: 0x2e
